In [1]:
import uuid
import os
import asyncio
from pprint import pp
from dotenv import load_dotenv
from tempfile import mkdtemp

from jobs import Worker
from output_data import run_sbml_amici, run_sbml_copasi, run_sbml_tellurium
from shared import MongoDbConnector, save_uploaded_file, upload_blob

load_dotenv('../assets/.env_dev')

MONGO_URI = os.getenv('MONGO_URI')
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
BUCKET_NAME = os.getenv('BUCKET_NAME')
DB_NAME = os.getenv('DB_NAME')

db_connector = MongoDbConnector(connection_uri=MONGO_URI, database_id=DB_NAME, connector_id="test_worker")

In [2]:
sbml_fp = "../model-examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml"

os.path.exists(sbml_fp)

True

In [3]:
params = {
    'sbml_fp': sbml_fp,
    'dur': 10,
    'n_steps': 100
}

amici_results = run_sbml_amici(**params)
copasi_results = run_sbml_copasi(**params)
tellurium_results = run_sbml_tellurium(**params)


2024-07-28 01:43:44.213 - amici.sbml_import - INFO - Finished importing SBML                         (2.22E-02s)
2024-07-28 01:43:44.240 - amici.sbml_import - INFO - Finished processing SBML observables            (2.53E-02s)
2024-07-28 01:43:44.242 - amici.sbml_import - INFO - Finished processing SBML event observables      (8.75E-07s)
2024-07-28 01:43:44.262 - amici.de_model - INFO - Finished computing xdot                            (1.58E-03s)
2024-07-28 01:43:44.265 - amici.de_model - INFO - Finished computing x0                              (1.18E-03s)
2024-07-28 01:43:44.276 - amici.de_model - INFO - Finished computing w                               (8.67E-03s)
2024-07-28 01:43:44.740 - amici.de_export - INFO - Finished generating cpp code                      (4.61E-01s)
2024-07-28 01:43:53.906 - amici.de_export - INFO - Finished compiling cpp code                       (9.16E+00s)


In [4]:
output = {'amici': amici_results, 'copasi': copasi_results, 'tellurium': tellurium_results}

In [5]:
output

{'amici': {'LacI protein': array([  0.        ,   1.04606259,   4.12384821,   9.12953655,
          15.92445756,  24.32648089,  34.11988472,  45.07534031,
          56.96930795,  69.59709928,  82.77904466,  96.36167147,
         110.2160732 , 124.23505983, 138.33001662, 152.42793368,
         166.46878916, 180.40333245, 194.19123629, 207.79957996,
         221.20160125, 234.37567963, 247.30449545, 259.97434272,
         272.37455494, 284.49703934, 296.33587614, 307.88699113,
         319.14787944, 330.11736899, 340.79542423, 351.1829753 ,
         361.28177461, 371.094273  , 380.62351334, 389.87303872,
         398.84681199, 407.54915174, 415.98466598, 424.15820752,
         432.07482196, 439.7397139 , 447.15821264, 454.33573615,
         461.27777287, 467.98985249, 474.4775287 , 480.74636646,
         486.80191906, 492.6497165 , 498.29525765, 503.74399794,
         509.00134113, 514.07263155, 518.96314835, 523.6780995 ,
         528.22261767, 532.60175566, 536.82048382, 540.88368751,


In [6]:
def output_stack(spec_name: str):
    stack = []
    for simulator_name, simulator_output in output.items():
        spec_data = simulator_output[spec_name]
        stack.append(spec_data)
    return stack 
        

In [7]:
output_stack("cI mRNA")

[array([0.        , 2.85494179, 5.00435142, 6.34482551, 7.09921386,
        7.4921065 , 7.66922871, 7.71593383, 7.68184878, 7.59649974,
        7.47803362, 7.33799025, 7.18394851, 7.02103276, 6.85279492,
        6.68175009, 6.5097088 , 6.33799227, 6.16757364, 5.99917395,
        5.83332823, 5.67043231, 5.51077647, 5.35457007, 5.20195993,
        5.05304386, 4.90788136, 4.76650165, 4.62890985, 4.49509208,
        4.36501922, 4.23865018, 4.11593432, 3.99681352, 3.88122379,
        3.76909669, 3.66036039, 3.55494054, 3.45276113, 3.35374495,
        3.25781429, 3.16489118, 3.07489784, 2.98775699, 2.903392  ,
        2.82172718, 2.74268789, 2.66620063, 2.59219321, 2.52059483,
        2.45133613, 2.38434922, 2.31956773, 2.25692685, 2.19636332,
        2.13781546, 2.08122316, 2.02652787, 1.97367259, 1.92260189,
        1.87326183, 1.82559999, 1.77956545, 1.73510871, 1.69218173,
        1.65073786, 1.61073183, 1.57211971, 1.53485889, 1.49890805,
        1.46422712, 1.43077725, 1.3985208 , 1.36

In [8]:
from jobs import Worker

In [9]:
worker = Worker({})

In [10]:
ci_mrna_comparison = worker.generate_utc_species_comparison(sbml_fp, 10, 100, 'cI mRNA')

2024-07-28 01:34:19.352 - amici.sbml_import - INFO - Finished loading SBML                           (4.68E-03s)
2024-07-28 01:34:19.376 - amici.sbml_import - INFO - Finished importing SBML                         (2.17E-02s)
2024-07-28 01:34:19.388 - amici.sbml_import - INFO - Finished processing SBML observables            (1.06E-02s)
2024-07-28 01:34:19.390 - amici.sbml_import - INFO - Finished processing SBML event observables      (8.34E-07s)
2024-07-28 01:34:19.407 - amici.de_model - INFO - Finished computing xdot                            (1.46E-03s)
2024-07-28 01:34:19.410 - amici.de_model - INFO - Finished computing x0                              (1.02E-03s)
2024-07-28 01:34:19.417 - amici.de_model - INFO - Finished computing w                               (5.66E-03s)
2024-07-28 01:34:19.743 - amici.de_export - INFO - Finished generating cpp code                      (3.23E-01s)
2024-07-28 01:34:29.102 - amici.de_export - INFO - Finished compiling cpp code                  

In [7]:
from output_data import get_sbml_species_mapping


def generate_sbml_utc_comparison(sbml_fp, dur, n_steps, worker):
    species_mapping = get_sbml_species_mapping(sbml_fp)
    results = {}
    for species_name in species_mapping.keys():
        species_comparison = worker.generate_utc_species_comparison(sbml_fp, dur, n_steps, species_name)
        results[species_name] = species_comparison
    
    return results
    

generate_sbml_utc_comparison(sbml_fp, 10, 100, Worker({}))

2024-07-28 01:46:05.220 - amici.sbml_import - INFO - Finished loading SBML                           (5.93E-03s)
2024-07-28 01:46:05.250 - amici.sbml_import - INFO - Finished importing SBML                         (2.74E-02s)
2024-07-28 01:46:05.263 - amici.sbml_import - INFO - Finished processing SBML observables            (1.08E-02s)
2024-07-28 01:46:05.266 - amici.sbml_import - INFO - Finished processing SBML event observables      (7.08E-07s)
2024-07-28 01:46:05.285 - amici.de_model - INFO - Finished computing xdot                            (1.97E-03s)
2024-07-28 01:46:05.290 - amici.de_model - INFO - Finished computing x0                              (1.84E-03s)
2024-07-28 01:46:05.298 - amici.de_model - INFO - Finished computing w                               (6.56E-03s)
2024-07-28 01:46:05.753 - amici.de_export - INFO - Finished generating cpp code                      (4.52E-01s)
2024-07-28 01:46:15.015 - amici.de_export - INFO - Finished compiling cpp code                  

{'LacI protein': {'mse': {'amici': {'amici': 0.0,
    'copasi': 5.827119216590656e-08,
    'tellurium': 212300.9711820942},
   'copasi': {'amici': 5.827119216590656e-08,
    'copasi': 0.0,
    'tellurium': 212300.75883979013},
   'tellurium': {'amici': 212300.9711820942,
    'copasi': 212300.75883979013,
    'tellurium': 0.0}},
  'prox': {'amici': {'amici': True, 'copasi': True, 'tellurium': False},
   'copasi': {'amici': True, 'copasi': True, 'tellurium': False},
   'tellurium': {'amici': False, 'copasi': False, 'tellurium': True}},
  'output_data': {'amici': [0.0,
    1.046062587050764,
    4.123848205099809,
    9.129536549688757,
    15.924457558996782,
    24.326480890525367,
    34.11988472055915,
    45.075340313432264,
    56.96930794894419,
    69.59709928197717,
    82.7790446645416,
    96.36167147169961,
    110.21607320235152,
    124.23505983087172,
    138.330016623065,
    152.42793367763852,
    166.4687891633536,
    180.4033324494624,
    194.19123628851406,
    207.

In [12]:
ci_mrna_comparison

{'mse': {'amici': {'amici': 0.0,
   'copasi': 1.9030929413640523e-11,
   'tellurium': 2.3559561546935707e-10},
  'copasi': {'amici': 1.9030929413640523e-11,
   'copasi': 0.0,
   'tellurium': 1.2787645268879774e-10},
  'tellurium': {'amici': 2.3559561546935707e-10,
   'copasi': 1.2787645268879774e-10,
   'tellurium': 0.0}},
 'prox': {'amici': {'amici': True, 'copasi': True, 'tellurium': True},
  'copasi': {'amici': True, 'copasi': True, 'tellurium': True},
  'tellurium': {'amici': True, 'copasi': True, 'tellurium': True}},
 'output_data': {'amici': [0.0,
   2.8549417948016287,
   5.004351419479255,
   6.3448255145697745,
   7.099213857814507,
   7.492106497852835,
   7.669228705383784,
   7.715933833359352,
   7.681848777032514,
   7.596499739391653,
   7.4780336217600505,
   7.33799025344281,
   7.183948513060199,
   7.021032755053169,
   6.8527949155467365,
   6.681750087938989,
   6.509708800129758,
   6.33799226736959,
   6.16757364414939,
   5.9991739475659625,
   5.833328229387457

In [4]:
len(amici_results['cI mRNA'])

100

In [5]:
len(copasi_results['cI mRNA'])

100

In [6]:
len(tellurium_results['cI mRNA'])

100

In [8]:
copasi_results['cI mRNA'][-1], amici_results['cI mRNA'][-1]

(0.7888212240349317, 0.869004338654352)